In [1]:
from shared_code.utility.spark.set_environ import *
from shared_code.utility.schemas.spark_table_schema import image_table_schema

set_azure_env()
from shared_code.utility.storage.table import TableAdapter
spark = get_session("add-thumbnails")

In [2]:
spark_df = spark.createDataFrame(spark.read.parquet("D:\\data\\processed\\reddit_images.parquet").collect(), schema=image_table_schema)

print(f"Total Number Of Records From Pre-Processing {spark_df.count()}")

display(spark_df.limit(10).toPandas())

Total Number Of Records From Pre-Processing 7190


,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,11094it,D:\data\images\greentext\q0rxrphnuqha1.jpg,Anon is insulinpilled,11094it,TheGourmetGaylord,https://i.redd.it/q0rxrphnuqha1.jpg,None,/r/greentext/comments/11094it/anon_is_insulinp...,d39377ac12d266df304429d5b03ff483,greentext,None,True,q0rxrphnuqha1.jpg,None,/data/images/greentext/thumbnail/q0rxrphnuqha1...,False
1,training,110961d,D:\data\images\CityPorn\2czgh0e5vqha1.jpg,St Paul’s Cathedral. London.,110961d,OkCardiologist2765,https://i.redd.it/2czgh0e5vqha1.jpg,NaN,/r/CityPorn/comments/110961d/st_pauls_cathedra...,ef3c3f82ec490f8d3ccc06948258a3b4,CityPorn,A man in a red coat is standing in front of a ...,True,2czgh0e5vqha1.jpg,a double decker bus is driving down the street,D:\data\images\CityPorn\thumbnail\2czgh0e5vqha...,False
2,training,1109a10,D:\data\images\SFWRedheads\hy9j0og8wqha1.jpg,Do you like my natural red curls?!,1109a10,ketunicorn,https://i.redd.it/hy9j0og8wqha1.jpg,NaN,/r/SFWRedheads/comments/1109a10/do_you_like_my...,9008fe70d17b904fcfc7c6a2232ad569,SFWRedheads,A girl in a pink dress is standing in front of...,True,hy9j0og8wqha1.jpg,a woman in a bikini standing next to a tree,D:\data\images\SFWRedheads\thumbnail\hy9j0og8w...,False
3,training,1109ycm,D:\data\images\CityPorn\7beed5n54rha1.jpg,"Taipei, the Seoul of Taiwan (which makes Kaohs...",1109ycm,Ducky118,https://i.redd.it/7beed5n54rha1.jpg,NaN,/r/CityPorn/comments/1109ycm/taipei_the_seoul_...,848b795097dca410c409797c679ab4fa,CityPorn,A bird flies over the water.,True,7beed5n54rha1.jpg,a cityscape of a city with tall buildings,D:\data\images\CityPorn\thumbnail\7beed5n54rha...,False
4,training,110b0wt,D:\data\images\SFWNextDoorGirls\6piZs7j.jpg,"A real no makeup moment, cause this is how Sun...",110b0wt,princessxo699,https://i.imgur.com/6piZs7j.jpg,Feelin Happy 😃,/r/SFWNextDoorGirls/comments/110b0wt/a_real_no...,5ccd774afb17fb0c44460121ed9ce61d,SFWNextDoorGirls,A girl in a pink hat and sunglasses is talking...,True,6piZs7j.jpg,a woman with glasses and a cat in her lap,D:\data\images\SFWNextDoorGirls\thumbnail\6piZ...,False
5,training,110b1an,D:\data\images\CityPorn\rgotpnatzpha1.jpg,"Duisburg, Detroit of Germany",110b1an,Expensive-Team7416,https://i.redd.it/rgotpnatzpha1.jpg,NaN,/r/CityPorn/comments/110b1an/duisburg_detroit_...,3eb930bdf3fac4e5b16617edad9df65a,CityPorn,A bird flies over the water.,True,rgotpnatzpha1.jpg,a large building with a clock on top of it,D:\data\images\CityPorn\thumbnail\rgotpnatzpha...,False
6,training,110bipg,D:\data\images\SFWNextDoorGirls\xywgncrbnrha1.jpg,enjoy your Sunday!,110bipg,trickyalice,https://i.redd.it/xywgncrbnrha1.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/110bipg/enjoy_you...,9475adbe05427c47772346f8ab672952,SFWNextDoorGirls,A woman in a pink shirt and jeans is talking t...,True,xywgncrbnrha1.jpg,a woman holding a cell phone in front of her face,D:\data\images\SFWNextDoorGirls\thumbnail\xywg...,False
7,training,110c2gi,D:\data\images\SFWRedheads\99hm504mtrha1.jpg,Happy sunday! Sending greetings! ❤️,110c2gi,GingeMariia,https://i.redd.it/99hm504mtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2gi/happy_sunday_s...,20ced2827eb9e9cd3464835306cfd075,SFWRedheads,A girl in a pink shirt and blue jeans is stand...,True,99hm504mtrha1.jpg,a woman taking a picture of herself in a bathr...,D:\data\images\SFWRedheads\thumbnail\99hm504mt...,False
8,training,110c2vo,D:\data\images\SFWRedheads\kkerxboqtrha1.jpg,The kind of girl you can have a beer with.,110c2vo,MissKatiesss,https://i.redd.it/kkerxboqtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2vo/the_kind_of_gi...,f53572296a0b8fb6654808866440316a,SFWRedheads,A woman in a red dress is talking to a man in ...,True,kkerxboqtrha1.jpg,a woman holding a bottle of beer next to a wall,D:\data\images\SFWRedheads\thumbnail\kkerxboqt...,False
9,training,110c9o8,D:\data\images\CityPorn\ndc1hr2wvrha1.jpg,"Quetta, the Tucson of Pakistan",110c9

In [3]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import PIL.Image
import logging

def resize_image(path: str):
	img = PIL.Image.open(path)
	try:
		copied_image = img.copy()
		i = copied_image.resize((512, 512))
		return i
	finally:
		img.close()

@udf(returnType=StringType())
def get_resized_image(big_image_path, subreddit, image_name):
	try:
		expected_path = os.path.join("D:", os.sep, "data", "images", subreddit, "thumbnail", image_name)
		if os.path.exists(expected_path):
			return expected_path
		else:
			logging.info(f"Resizing image {big_image_path} to {expected_path}...")
			r = resize_image(big_image_path)
			r.save(expected_path)
			r.close()
			return expected_path
	except Exception as e:
		logging.error(f"Error resizing image {big_image_path}...")
		return ""

In [4]:
result = spark_df \
	.withColumn("small_image", get_resized_image("image", "subreddit", "image_name")).collect()

print(f"Total Number of Records Updated To Write: {len(result)}")

writeable_df = spark.createDataFrame(result, schema=image_table_schema)

display(writeable_df.toPandas())

records = writeable_df.toPandas().to_dict(orient="records")

table_adapter = TableAdapter()

for record in records:
	table_adapter.upsert_entity_to_table("training", record)

refreshed = list(table_adapter.get_all_entities("training"))

writeable_df = spark.createDataFrame(refreshed, schema=image_table_schema)

display(writeable_df.limit(100).toPandas())

writeable_df.write.parquet("D:\\data\\processed\\reddit_images_processed.parquet", mode="overwrite")

Total Number of Records Updated To Write: 7190


,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,11094it,D:\data\images\greentext\q0rxrphnuqha1.jpg,Anon is insulinpilled,11094it,TheGourmetGaylord,https://i.redd.it/q0rxrphnuqha1.jpg,None,/r/greentext/comments/11094it/anon_is_insulinp...,d39377ac12d266df304429d5b03ff483,greentext,None,True,q0rxrphnuqha1.jpg,None,D:\data\images\greentext\thumbnail\q0rxrphnuqh...,False
1,training,110961d,D:\data\images\CityPorn\2czgh0e5vqha1.jpg,St Paul’s Cathedral. London.,110961d,OkCardiologist2765,https://i.redd.it/2czgh0e5vqha1.jpg,NaN,/r/CityPorn/comments/110961d/st_pauls_cathedra...,ef3c3f82ec490f8d3ccc06948258a3b4,CityPorn,A man in a red coat is standing in front of a ...,True,2czgh0e5vqha1.jpg,a double decker bus is driving down the street,D:\data\images\CityPorn\thumbnail\2czgh0e5vqha...,False
2,training,1109a10,D:\data\images\SFWRedheads\hy9j0og8wqha1.jpg,Do you like my natural red curls?!,1109a10,ketunicorn,https://i.redd.it/hy9j0og8wqha1.jpg,NaN,/r/SFWRedheads/comments/1109a10/do_you_like_my...,9008fe70d17b904fcfc7c6a2232ad569,SFWRedheads,A girl in a pink dress is standing in front of...,True,hy9j0og8wqha1.jpg,a woman in a bikini standing next to a tree,D:\data\images\SFWRedheads\thumbnail\hy9j0og8w...,False
3,training,1109ycm,D:\data\images\CityPorn\7beed5n54rha1.jpg,"Taipei, the Seoul of Taiwan (which makes Kaohs...",1109ycm,Ducky118,https://i.redd.it/7beed5n54rha1.jpg,NaN,/r/CityPorn/comments/1109ycm/taipei_the_seoul_...,848b795097dca410c409797c679ab4fa,CityPorn,A bird flies over the water.,True,7beed5n54rha1.jpg,a cityscape of a city with tall buildings,D:\data\images\CityPorn\thumbnail\7beed5n54rha...,False
4,training,110b0wt,D:\data\images\SFWNextDoorGirls\6piZs7j.jpg,"A real no makeup moment, cause this is how Sun...",110b0wt,princessxo699,https://i.imgur.com/6piZs7j.jpg,Feelin Happy 😃,/r/SFWNextDoorGirls/comments/110b0wt/a_real_no...,5ccd774afb17fb0c44460121ed9ce61d,SFWNextDoorGirls,A girl in a pink hat and sunglasses is talking...,True,6piZs7j.jpg,a woman with glasses and a cat in her lap,D:\data\images\SFWNextDoorGirls\thumbnail\6piZ...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7185,training,zoqw3x,D:\data\images\HotGirlNextDoor\mjwdnicsum6a1.jpg,Curvy(iktr),zoqw3x,awd2009,https://i.redd.it/mjwdnicsum6a1.jpg,HotGirlNextDoor,/r/HotGirlNextDoor/comments/zoqw3x/curvyiktr/,a507218fb1b9906946ef6583151ae919,HotGirlNextDoor,A woman in a white shirt and black pants is st...,True,mjwdnicsum6a1.jpg,a woman standing in front of a fireplace in a ...,D:\data\images\HotGirlNextDoor\thumbnail\mjwdn...,False
7186,training,zor18g,D:\data\images\SFWNextDoorGirls\o2i3ryo7wm6a1.jpg,Girls night out… did I catch you peaking?,zor18g,xxkatyaxxfr,https://i.redd.it/o2i3ryo7wm6a1.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/zor18g/girls_nigh...,df23935a2a37f5d00f32c0d404c4e4af,SFWNextDoorGirls,A little girl in a pink dress is sitting on a ...,True,o2i3ryo7wm6a1.jpg,a woman in a bikini sitting on a bed,D:\data\images\SFWNextDoorGirls\thumbnail\o2i3...,False
7187,training,zorm56,D:\data\images\SFWRedheads\etue4ebd2n6a1.jpg,Getting into the Christmas spirit 🎄,zorm56,JessicaRabbitOz,https://i.redd.it/etue4ebd2n6a1.jpg,NaN,/r/SFWRedheads/comments/zorm56/getting_into_th...,c0f2a3312ef8cff6b48f0237efb196bb,SFWRedheads,A little girl in a pink dress is sitting on a ...,True,etue4ebd2n6a1.jpg,a woman is taking a picture of herself in the ...,D:\data\images\SFWRedheads\thumbnail\etue4ebd2...,False
7188,training,zos5uz,D:\data\images\SFWNextDoorGirls\ubz0huhg8n6a1.jpg,Taco Night over here!,zos5uz,Iambatmansmom38,https://i.redd.it/ubz0huhg8n6a1.jpg,Feelin Happy 😃,/r/SFWNextDoorGirls/comments/zos5uz/taco_night...,014f6ff9583fe52c4cf1c7eacf067bb5,SFWNextDoorGirls,A woman in a pink shirt and blue jeans is talk...,True,ubz0huhg8n6a1.jpg,a woman standing in a kitchen preparing food,D:\data\images\SFWNextDoorGirls\thumbnail\ubz

In [5]:
new_frame = spark.createDataFrame(spark.read.parquet("D:\\data\\processed\\reddit_images_processed.parquet").collect(), schema=image_table_schema)

print("== Finished Adding Thumbnails ==")
display(new_frame.limit(100).toPandas())

,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,11094it,D:\data\images\greentext\q0rxrphnuqha1.jpg,Anon is insulinpilled,11094it,TheGourmetGaylord,https://i.redd.it/q0rxrphnuqha1.jpg,None,/r/greentext/comments/11094it/anon_is_insulinp...,d39377ac12d266df304429d5b03ff483,greentext,None,True,q0rxrphnuqha1.jpg,None,D:\data\images\greentext\thumbnail\q0rxrphnuqh...,False
1,training,110961d,D:\data\images\CityPorn\2czgh0e5vqha1.jpg,St Paul’s Cathedral. London.,110961d,OkCardiologist2765,https://i.redd.it/2czgh0e5vqha1.jpg,NaN,/r/CityPorn/comments/110961d/st_pauls_cathedra...,ef3c3f82ec490f8d3ccc06948258a3b4,CityPorn,A man in a red coat is standing in front of a ...,True,2czgh0e5vqha1.jpg,a double decker bus is driving down the street,D:\data\images\CityPorn\thumbnail\2czgh0e5vqha...,False
2,training,1109a10,D:\data\images\SFWRedheads\hy9j0og8wqha1.jpg,Do you like my natural red curls?!,1109a10,ketunicorn,https://i.redd.it/hy9j0og8wqha1.jpg,NaN,/r/SFWRedheads/comments/1109a10/do_you_like_my...,9008fe70d17b904fcfc7c6a2232ad569,SFWRedheads,A girl in a pink dress is standing in front of...,True,hy9j0og8wqha1.jpg,a woman in a bikini standing next to a tree,D:\data\images\SFWRedheads\thumbnail\hy9j0og8w...,False
3,training,1109ycm,D:\data\images\CityPorn\7beed5n54rha1.jpg,"Taipei, the Seoul of Taiwan (which makes Kaohs...",1109ycm,Ducky118,https://i.redd.it/7beed5n54rha1.jpg,NaN,/r/CityPorn/comments/1109ycm/taipei_the_seoul_...,848b795097dca410c409797c679ab4fa,CityPorn,A bird flies over the water.,True,7beed5n54rha1.jpg,a cityscape of a city with tall buildings,D:\data\images\CityPorn\thumbnail\7beed5n54rha...,False
4,training,110b0wt,D:\data\images\SFWNextDoorGirls\6piZs7j.jpg,"A real no makeup moment, cause this is how Sun...",110b0wt,princessxo699,https://i.imgur.com/6piZs7j.jpg,Feelin Happy 😃,/r/SFWNextDoorGirls/comments/110b0wt/a_real_no...,5ccd774afb17fb0c44460121ed9ce61d,SFWNextDoorGirls,A girl in a pink hat and sunglasses is talking...,True,6piZs7j.jpg,a woman with glasses and a cat in her lap,D:\data\images\SFWNextDoorGirls\thumbnail\6piZ...,False
5,training,110b1an,D:\data\images\CityPorn\rgotpnatzpha1.jpg,"Duisburg, Detroit of Germany",110b1an,Expensive-Team7416,https://i.redd.it/rgotpnatzpha1.jpg,NaN,/r/CityPorn/comments/110b1an/duisburg_detroit_...,3eb930bdf3fac4e5b16617edad9df65a,CityPorn,A bird flies over the water.,True,rgotpnatzpha1.jpg,a large building with a clock on top of it,D:\data\images\CityPorn\thumbnail\rgotpnatzpha...,False
6,training,110bipg,D:\data\images\SFWNextDoorGirls\xywgncrbnrha1.jpg,enjoy your Sunday!,110bipg,trickyalice,https://i.redd.it/xywgncrbnrha1.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/110bipg/enjoy_you...,9475adbe05427c47772346f8ab672952,SFWNextDoorGirls,A woman in a pink shirt and jeans is talking t...,True,xywgncrbnrha1.jpg,a woman holding a cell phone in front of her face,D:\data\images\SFWNextDoorGirls\thumbnail\xywg...,False
7,training,110c2gi,D:\data\images\SFWRedheads\99hm504mtrha1.jpg,Happy sunday! Sending greetings! ❤️,110c2gi,GingeMariia,https://i.redd.it/99hm504mtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2gi/happy_sunday_s...,20ced2827eb9e9cd3464835306cfd075,SFWRedheads,A girl in a pink shirt and blue jeans is stand...,True,99hm504mtrha1.jpg,a woman taking a picture of herself in a bathr...,D:\data\images\SFWRedheads\thumbnail\99hm504mt...,False
8,training,110c2vo,D:\data\images\SFWRedheads\kkerxboqtrha1.jpg,The kind of girl you can have a beer with.,110c2vo,MissKatiesss,https://i.redd.it/kkerxboqtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2vo/the_kind_of_gi...,f53572296a0b8fb6654808866440316a,SFWRedheads,A woman in a red dress is talking to a man in ...,True,kkerxboqtrha1.jpg,a woman holding a bottle of beer next to a wall,D:\data\images\SFWRedheads\thumbnail\kkerxboqt...,False
9,training,110c9o8,D:\data\images\CityPorn\ndc1hr2wvrha1.jpg,"Quetta, the Tucson of Pakistan",110c9